# ----------------------------------- Projeto Cetus --------------------------------------------


### Assunto: Orçamento da Despesa Pública
### Defasagem: D-2

In [125]:
# Autor: Bruno Jardim
# Data: 10/02/2019
# Descrição: Código para salvar os dados do portal transparência no datalake (AWS - S3)
# URL: http://www.portaltransparencia.gov.br/download-de-dados

In [34]:
def CetusLakeOrcamentoDespesasGov(dataproc,ANO_INFO,exec_lake = 'prod'):
    
    #---- Pagina de Extracao -----------
    # http://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/
    
    #---- Metadados do Arquivo ---------
    # http://www.portaltransparencia.gov.br/pagina-interna/603417-dicionario-de-dados-orcamento-da-despesa
    
    
    # ----- Pegando a Data da Informacao e Gravando na Tabela -------------------
    import requests
    from bs4 import BeautifulSoup
    import re
    
    if exec_lake == 'prod':
        print('CETUS --> Executando em Producao')
        extr = 'P'
        # Utilizado para producao (execucao semanal ou mensal)
        URL = 'http://www.portaltransparencia.gov.br/orcamento/despesas?ordenarPor=ano&direcao=desc'
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        tableDiv = soup.find_all('div', id="datas")
        datinfo = re.sub("[^\d\.]", "", str(tableDiv))[2:]
        #print('A data da informação é: ', datinfo)
        # ----------------------------------------------------------------------------
    else:
        print('CETUS --> Executando Carga Historica')
        # Formato que pegamos no site quando no modo producao (DDMMAAAA)
        datinfo = '31' + '12' + str(ANO_INFO)
        extr = 'H'
        
    
   
    import urllib.request
    urllib.request.urlretrieve('http://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/'+str(ANO_INFO), 'dados_desp_' + str(ANO_INFO) + '.zip')

    import zipfile
    with zipfile.ZipFile('dados_desp_' + str(ANO_INFO) + '.zip',"r") as zip_ref:
        zip_ref.extractall('dados_desp_' + str(ANO_INFO))

    import pandas as pd
    import chardet
    with open('dados_desp_' + str(ANO_INFO) +'/'+str(ANO_INFO)+'_OrcamentoDespesa.zip.csv', 'rb') as f:
        result = chardet.detect(f.read())  # or readline if the file is large
    df = pd.read_csv('dados_desp_' + str(ANO_INFO) +'/'+str(ANO_INFO)+'_OrcamentoDespesa.zip.csv', encoding=result['encoding'], error_bad_lines=False,sep=';', engine='python')
    df['DT_INFO'] = datinfo
    df['DT_DATAPROC'] = dataproc
    df['ID_PROC'] = extr
    return df


def CetusSalvaCSVS3(dataframe,nm_s3_file,s3_path):

    import s3fs
    S3fs = s3fs.S3FileSystem()

    # Nome do arquivo a ser salvo no S3
    #nm_s3_file = 'OrcamentoDespesas_'+str(dtref)+'.csv'

    # Caminho no lake (S3)
    #s3_path = 'projeto-cetus/datalake/OrcamentoDespesas/dados/'

    bytes_to_write = dataframe.to_csv(sep = ',',header=True, index=False, index_label=None).encode()
    with S3fs.open(s3_path+nm_s3_file, 'wb') as f:
        f.write(bytes_to_write)
    return print('Salvo com Sucesso :)')



def CetusReadS3CSVFile(nm_bucket,path_file,sep=','):
    import boto3
    bucket=nm_bucket # Or whatever you called your bucket
    data_key = path_file # Where the file is within your bucket
    data_location = 's3://{}/{}'.format(bucket, data_key)
    try:
        df = pd.read_csv(data_location,sep=sep,encoding='utf-8',error_bad_lines=False) 

    except:
        df = pd.read_csv(data_location,sep=sep,encoding='latin-1',error_bad_lines=False)

    return df



### Carga histórica de 2014 a 2018

In [11]:
# years = range(2014,2019)
# for ano in years:
#     print('Extraindo dados de: ',ano)
#     dados = CetusLakeOrcamentoDespesasGov(ano,exec_lake = 'Prod')
#     nm_s3_file = 'OP_Lake_OrcDesp_'+str(ano)+'.csv'
#     s3_path = 'projeto-bigdata-cetus/datalake/OrcamentoPublico/'
#     CetusSalvaCSVS3(dados,nm_s3_file,s3_path)

### Produção

In [38]:
import datetime
now = datetime.datetime.now() - datetime.timedelta(0,0,0,0,3)
anoref = now.strftime("%Y")
anomesdia = now.strftime("%d%m%Y")

print('Prod: ',anoref)
dados = CetusLakeOrcamentoDespesasGov(anomesdia,anoref,exec_lake = 'prod')
nm_s3_file = 'OP_Lake_OrcDesp_'+str(ano)+'.csv'
s3_path = 'projeto-bigdata-cetus/datalake/OrcamentoPublico/'
CetusSalvaCSVS3(dados,nm_s3_file,s3_path)

Prod:  2019
CETUS --> Executando em Producao


Skipping line 1716: ';' expected after '"'


Salvo com Sucesso :)


### Lendo dados Salvos no Lake

In [39]:
import pandas as pd
ano = 2019
nm_s3_file = 'OP_Lake_OrcDesp_'+str(ano)+'.csv'
df = CetusReadS3CSVFile('projeto-bigdata-cetus','datalake/OrcamentoPublico/'+nm_s3_file,sep=',')
df.shape

(21244, 27)

In [40]:
df.head()

,EXERCÍCIO,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO SUBORDINADO,NOME ÓRGÃO SUBORDINADO,CÓDIGO UNIDADE ORÇAMENTÁRIA,NOME UNIDADE ORÇAMENTÁRIA,CÓDIGO FUNÇÃO,NOME FUNÇÃO,CÓDIGO SUBFUNÇÃO,...,CÓDIGO GRUPO DE DESPESA,NOME GRUPO DE DESPESA,CÓDIGO ELEMENTO DE DESPESA,NOME ELEMENTO DE DESPESA,ORÇAMENTO INICIAL (R$),ORÇAMENTO ATUALIZADO (R$),ORÇAMENTO REALIZADO (R$),DT_INFO,DT_DATAPROC,ID_PROC
0,2019,3000,Tribunal de Contas da União,3000,Tribunal de Contas da União - Unidades com vín...,3101,TRIBUNAL DE CONTAS DA UNIAO,1,Legislativa,32,...,3,Outras Despesas Correntes,39,Outros Serviços de Terceiros - Pessoa Jurídica,",00",",00","2090,33",12022019,13022019,P
1,2019,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,3,Outras Despesas Correntes,0,Não informado,"332500,00",",00",",00",12022019,13022019,P
2,2019,20000,Presidência da República,24208,Instituto Nacional de Tecnologia da Informação,20204,INST.NAC.DE TECNOLOGIA DA INFORMACAO-ITI,4,Administração,125,...,3,Outras Despesas Correntes,0,Não informado,"9084508,52","9084508,52",",00",12022019,13022019,P
3,2019,20000,Presidência da República,24208,Instituto Nacional de Tecnologia da Informação,20204,INST.NAC.DE TECNOLOGIA DA INFORMACAO-ITI,4,Administração,125,...,3,Outras Despesas Correntes,40,Serviços de Tecnologia da Informação e Comunic...,"452000,00","452000,00",",00",12022019,13022019,P
4,2019,20000,Presidência da República,24208,Instituto Nacional de Tecnologia da Informação,20204,INST.NAC.DE TECNOLOGIA DA INFORMACAO-ITI,4,Administração,125,...,4,Investimentos,0,Não informado,"2660000,00","2660000,00",",00",12022019,13022019,P
